# Stencil Numba Serial

In [1]:
%%writefile st-nu-seq.py
import numpy as np
from numba import jit, config, prange
from time import time

config.DUMP_ASSEMBLY = 0
config.NUMBA_ENABLE_AVX = 1
config.NUMBA_NUM_THREADS = 1

@jit('(float64[:,:],float64[:,:])', nopython=True, parallel=True, nogil=True) 
def kernel_seq(anew, aold) :
    anew[1:-1, 1:-1] = ( aold[1:-1, 1:-1] * 0.5 + 
                       ( aold[2:  , 1:-1] + aold[ :-2, 1:-1] +
                         aold[1:-1, 2:  ] + aold[1:-1,  :-2] ) * 0.125 )

n            = 4800    # nxn grid (4800,1,500)=1500; (4800,1,5)=12
energy       = 1.0     # energy to be injected per iteration
niters       = 500     # number of iterations
nsources     = 3       # sources of energy
size         = n + 2
sizeEnd      = n + 1
heat         = np.zeros((1), np.float64)     # system total heat
anew         = np.zeros((size,  size), np.float64)
aold         = np.zeros((size,  size), np.float64)
sources      = np.empty((nsources, 2), np.int16)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
niters       = (niters + 1) // 2

t0 = time()
for iters in range(niters) :
    kernel_seq(anew, aold)
    for i in range(nsources) :
        anew[sources[i, 0], sources[i, 1]] += energy
    kernel_seq(aold, anew)
    for i in range(nsources) :
        aold[sources[i, 0], sources[i, 1]] += energy

heat[0] = np.sum( aold[1:-1, 1:-1] )  # system total heat
t0 = time() - t0

print("Heat = %0.4f | Tempo = %0.4f | Thread count = %s" %
      (heat[0], t0, config.NUMBA_NUM_THREADS))

Overwriting st-nu-seq.py


In [2]:
%%bash
a='st-nu-seq.py'
s='/prj/ampemi/xxxx.xxxx/stnc/Numba'
d='/scratch/ampemi/xxxx.xxxx/stnc/Numba'
cp  $s/$a  $d
ls -lh $d/$a

-rw-r--r-- 1 xxxx.xxxx ampemi 1.5K Nov 24 20:24 /scratch/ampemi/xxxx.xxxx/stnc/Numba/st-nu-seq.py


In [3]:
%%writefile st-nu-seq.srm
#!/bin/bash
#SBATCH --ntasks=1             # Total tasks(CPUs)
#SBATCH -p cpu_small           # Select partition
#SBATCH -J stnuseq             # Job name
#SBATCH --time=00:05:00        # Limit execution time

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- Number of nodes:' $SLURM_JOB_NUM_NODES
echo '- Total tasks:' $SLURM_NTASKS
echo '- Nodes alocated:' $SLURM_JOB_NODELIST
echo '- Directory where sbatch was called ($SLURM_SUBMIT_DIR):'
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR
nodeset -e $SLURM_JOB_NODELIST

# Working dir
cd /scratch/ampemi/xxxx.xxxx/stnc/Numba

# Module
module load anaconda3/2018.12

# Executable
EXEC='python st-nu-seq.py'

# RUN
echo '-- srun -------------------------------'
echo '$ srun --mpi=pmi2 -n' $SLURM_NTASKS $EXEC
srun --mpi=pmi2 -n $SLURM_NTASKS $EXEC
echo '-- END --------------------------------'

Overwriting st-nu-seq.srm


<hr style="height:10px;border-width:0;background-color:green">

In [4]:
%%bash
sbatch st-nu-seq.srm

Submitted batch job 781495


In [5]:
! squeue -n stnuseq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            781495 cpu_small  stnuseq xxxx. PD       0:00      1 (Priority)


In [9]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Numba'
cat $d/slurm-781495.out

- Job ID: 781495
- Tasks per node:
- Number of nodes: 1
- Total tasks: 1
- Nodes alocated: sdumont1279
- Directory where sbatch was called ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
sdumont1279
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 python st-nu-seq.py
Heat = 1500.0000 | Tempo = 30.3982 | Thread count = 1
-- END --------------------------------


In [1]:
%%bash
sbatch st-nu-seq.srm
sbatch st-nu-seq.srm

Submitted batch job 788104
Submitted batch job 788105


In [2]:
! squeue -n stnuseq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788104 cpu_small  stnuseq xxxx. PD       0:00      1 (Priority)
            788105 cpu_small  stnuseq xxxx. PD       0:00      1 (Priority)


In [2]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Numba'
cat $d/slurm-788104.out
cat $d/slurm-788105.out

- Job ID: 788104
- Tasks per node:
- Number of nodes: 1
- Total tasks: 1
- Nodes alocated: sdumont1149
- Directory where sbatch was called ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
sdumont1149
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 python st-nu-seq.py
Heat = 1500.0000 | Tempo = 30.5014 | Thread count = 1
-- END --------------------------------
- Job ID: 788105
- Tasks per node:
- Number of nodes: 1
- Total tasks: 1
- Nodes alocated: sdumont1149
- Directory where sbatch was called ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
sdumont1149
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 python st-nu-seq.py
Heat = 1500.0000 | Tempo = 30.5266 | Thread count = 1
-- END --------------------------------


<hr style="height:10px;border-width:0;background-color:red">

## Version

In [1]:
! python --version

Python 3.7.3


In [2]:
import numba
print(numba.__version__)

0.41.0


In [4]:
import numpy
numpy.show_config()

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
